# Content Change Detection (Repository Pattern)

**New repository-based change detection using Phase 4 migration pattern**

## Overview

This notebook demonstrates the **new repository pattern** for change detection:
1. Load modified content from `content_repo`
2. Chunk modified content and store in `content_chunks`
3. Detect changes using `ChecksumChangeDetector` with `AuditRepository`
4. Store results in `content_change_log`

### Key Benefits

- ✅ **Database-Agnostic**: Works with SQLite (local) and Databricks (production)
- ✅ **Clean API**: All operations through repository pattern
- ✅ **Auto-Detection**: Automatically detects environment
- ✅ **Simpler Code**: No custom adapter classes needed

### Architecture

```
ChecksumChangeDetector
    ↓
AuditRepository (business logic)
    ↓
Backend (SQLiteBackend or SparkBackend)
    ↓
Database (SQLite or Databricks Unity Catalog)
```

---

## Step 1: Setup and Configuration

In [1]:
# Configuration
import sys
import os
from pathlib import Path
from datetime import datetime
import uuid

# Add project root to path
NOTEBOOK_DIR = Path(os.getcwd())
PROJECT_ROOT = NOTEBOOK_DIR if NOTEBOOK_DIR.name == "faq_update" else NOTEBOOK_DIR
sys.path.insert(0, str(PROJECT_ROOT))

# Import environment utilities
from utility.environment import is_databricks

# Detect environment
IS_DATABRICKS = is_databricks()

if IS_DATABRICKS:
    # Databricks configuration
    dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
    dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
    dbutils.widgets.text("volume", "datacloud-dita-gen-volume", "Volume")
    
    CATALOG = dbutils.widgets.get("catalog")
    SCHEMA = dbutils.widgets.get("schema")
    VOLUME = dbutils.widgets.get("volume")
    
    PROJECT_ROOT = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update"
    
    print(f"🏢 Databricks mode: {CATALOG}.{SCHEMA}")
    print(f"📁 Project root: {PROJECT_ROOT}")
else:
    # Local configuration
    CATALOG = None
    SCHEMA = None
    VOLUME = None
    
    DATABASE_DIR = PROJECT_ROOT / "databases"
    DB_PATH = str(DATABASE_DIR / "faq_update.db")
    
    print(f"💻 Local mode")
    print(f"📁 Database: {DB_PATH}")
    print(f"📁 Project root: {PROJECT_ROOT}")

💻 Local mode
📁 Database: c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db
📁 Project root: c:\Users\praka\projects\FAQ_combined\faq_update


## Step 2: Initialize Backend and Repository

**This is the key difference:** We use the `BackendFactory` and `AuditRepository` instead of custom adapters.

In [2]:
from database.backends.factory import BackendFactory
from database.config import DatabaseConfig
from database.repository import AuditRepository, ContentRepository

print("="*80)
print("🔧 INITIALIZING DATABASE BACKEND AND REPOSITORIES")
print("="*80)

# Create configuration based on environment
if IS_DATABRICKS:
    config = DatabaseConfig(
        backend='databricks',
        catalog=CATALOG,
        schema=SCHEMA
    )
    print(f"✓ Configuration: Databricks ({CATALOG}.{SCHEMA})")
else:
    config = DatabaseConfig(
        backend='sqlite',
        db_path=DB_PATH
    )
    print(f"✓ Configuration: SQLite ({DB_PATH})")

# Create backend (auto-selects SQLite or Spark backend based on config)
backend = BackendFactory.create_backend(config)
print(f"✓ Backend created: {backend.__class__.__name__}")

# Create repositories
audit_repo = AuditRepository(backend)
content_repo = ContentRepository(backend)
print(f"✓ Repositories initialized")

print("="*80)
print("✅ Database backend and repositories ready!")
print("="*80)

2025-11-02 14:42:31.782 | INFO     | database.backends.factory:create_backend:133 - Creating backend: sqlite (sqlite://c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db)
2025-11-02 14:42:31.783 | DEBUG    | database.backends.sqlite.connection:__init__:90 - SQLiteConnectionManager initialized: db_path=c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db, timeout=30.0, journal_mode=WAL
2025-11-02 14:42:31.783 | INFO     | database.backends.sqlite.backend:__init__:128 - SQLiteBackend initialized: db_path=c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db
2025-11-02 14:42:31.784 | DEBUG    | database.backends.factory:create_backend:141 - Backend stored as singleton
2025-11-02 14:42:31.786 | DEBUG    | database.backends.sqlite.connection:get_connection:110 - Creating new SQLite connection to c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db
2025-11-02 14:42:31.788 | DEBUG    | database.backends.sqlite.connec

🔧 INITIALIZING DATABASE BACKEND AND REPOSITORIES
✓ Configuration: SQLite (c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db)
✓ Backend created: SQLiteBackend
✓ Repositories initialized
✅ Database backend and repositories ready!


## Step 3: Configure Detection Parameters

In [3]:
# Detection parameters
SINCE_DATE = "2025-11-02 10:24:29"  # Compare checksums before vs after this date
SIMILARITY_THRESHOLD = 0.8           # Threshold for modification detection (80%)
CHUNK_SIZE = 1000                    # Character size for chunks
DETECTION_RUN_ID = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:8]}"

print(f"\n{'='*80}")
print("🔍 DETECTION CONFIGURATION")
print(f"{'='*80}")
print(f"Environment: {'Databricks' if IS_DATABRICKS else 'SQLite'}")
print(f"Comparison: Before '{SINCE_DATE}' vs After '{SINCE_DATE}'")
print(f"Chunk Size: {CHUNK_SIZE} characters")
print(f"Similarity Threshold: {SIMILARITY_THRESHOLD}")
print(f"Detection Run ID: {DETECTION_RUN_ID}")
print(f"{'='*80}\n")


🔍 DETECTION CONFIGURATION
Environment: SQLite
Comparison: Before '2025-11-02 10:24:29' vs After '2025-11-02 10:24:29'
Chunk Size: 1000 characters
Similarity Threshold: 0.8
Detection Run ID: run_20251102_144231_400e5290



## Step 4: Load Modified Content from content_repo

Use `ContentRepository` to load modified files.

In [4]:
import pandas as pd

print("\n📊 Loading modified content...")

# Use ContentRepository to get modified files
modified_files = content_repo.get_files_modified_after(SINCE_DATE)

print(f"Found {len(modified_files)} modified files since {SINCE_DATE}")

if len(modified_files) > 0:
    # Convert to DataFrame for display
    modified_df = pd.DataFrame(modified_files)
    print("\nModified files:")
    display(modified_df[['ud_source_file_id', 'raw_file_nme', 'raw_file_version_nbr', 'last_modified_dt']].head(10))
else:
    print("\n⚠️ No modified files found. Adjust SINCE_DATE or check data ingestion.")

2025-11-02 14:42:31.817 | DEBUG    | database.repository.content_repository:get_files_modified_after:428 - Retrieving files modified after '2025-11-02 10:24:29' (status=Active)
2025-11-02 14:42:31.818 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 24 rows
2025-11-02 14:42:31.819 | DEBUG    | database.repository.base:execute_query:590 - Query returned 24 rows
2025-11-02 14:42:31.820 | DEBUG    | database.repository.content_repository:get_files_modified_after:455 - Found 24 files modified after '2025-11-02 10:24:29'



📊 Loading modified content...
Found 24 modified files since 2025-11-02 10:24:29

Modified files:


ud_source_file_id            raw_file_nme  raw_file_version_nbr  \
0                 15   Employee_Handbook.pdf                     2   
1                 16  IT_Security_Policy.pdf                     2   
2                 17  Remote_Work_Policy.pdf                     2   
3                 18       Travel_Policy.pdf                     2   
4                 19      Parental_Leave.pdf                     2   
5                 20    Workplace_Safety.pdf                     1   
6                 21    Health_Insurance.pdf                     2   
7                 22     Vacation_Policy.pdf                     2   
8                 23        Email_Policy.pdf                     2   
9                 24    Equipment_Return.pdf                     1   

             last_modified_dt  
0  2025-11-02 14:42:23.718163  
1  2025-11-02 14:42:23.718163  
2  2025-11-02 14:42:23.718163  
3  2025-11-02 14:42:23.718163  
4  2025-11-02 14:42:23.718163  
5  2025-11-02 14:42:23.718163  
6  2025-11-02 14:42:23.718163  
7  2025-11-02 14:42:23.718163  
8  2025-11-02 14:42:23.718163  
9  2025-11-02 14:42:23.718163

## Step 5: Chunk Modified Content and Store

Process files and store chunks in `content_chunks` table.

In [5]:
# Chunking configuration - Using constants from config module
from config.constants import DEFAULT_CHUNK_SIZE, DEFAULT_CHUNK_OVERLAP

CHUNK_SIZE = DEFAULT_CHUNK_SIZE      # 3000 characters (~750 tokens)
CHUNK_OVERLAP = DEFAULT_CHUNK_OVERLAP  # 600 characters (20% overlap)

In [6]:
print(f"\n{'='*80}")
print("  CHUNKING AND CHECKSUM COMPUTATION - HybridMarkdownChunker")
print(f"{'='*80}\n")

# Import required modules
from chunking import HybridMarkdownChunker
from data_ingestion import ChunkIngestion
from pathlib import Path

# Initialize chunker with production-recommended settings
chunker = HybridMarkdownChunker(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# Initialize chunk ingestion (uses same repository)
chunk_ingestion = ChunkIngestion(repository=content_repo)

# Process each file
files_processed = 0
total_chunks = 0
ingestion_results = []

for idx, row in modified_df.iterrows():
    # Build path to markdown file
    if is_databricks():
        # Databricks: Use absolute path from volume
        markdown_path = Path(f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update") / row['extracted_markdown_file_path']
    else:
        # Local: Use relative path
        markdown_path = Path(row['extracted_markdown_file_path'])
    
    if not markdown_path.exists():
        print(f"⚠️  WARNING: File not found: {markdown_path}")
        continue
    
    # Read content
    try:
        content = markdown_path.read_text(encoding='utf-8')
    except Exception as e:
        print(f"✗ ERROR reading {markdown_path}: {e}")
        continue
    
    # Chunk content with automatic checksum computation and metadata extraction
    # Returns: List of (chunk_text, checksum, ChunkMetadata) tuples
    try:
        chunks_with_checksums = chunker.chunk_with_checksums(content)
    except Exception as e:
        print(f"✗ ERROR chunking {row['raw_file_nme']}: {e}")
        ingestion_results.append({
            'file_id': row['ud_source_file_id'],
            'file_name': row['raw_file_nme'],
            'success': False,
            'chunks': 0,
            'error': f"Chunking error: {str(e)}"
        })
        continue
    
    # Ingest chunks using ChunkIngestion (handles clearing + insertion)
    try:
        result = chunk_ingestion.ingest_chunks_for_file(
            ud_source_file_id=row['ud_source_file_id'],
            chunks_with_checksums=chunks_with_checksums,
            clear_existing=True  # Clear old chunks before inserting new ones
        )
        
        if result.get('success'):
            files_processed += 1
            rows_inserted = result.get('rows_inserted', 0)
            total_chunks += rows_inserted
            print(f"  ✓ {row['raw_file_nme']:40s} → {rows_inserted:3d} chunks")
        else:
            print(f"  ✗ {row['raw_file_nme']:40s} → ERROR: {result.get('message', 'Unknown error')}")
        
        ingestion_results.append({
            'file_id': row['ud_source_file_id'],
            'file_name': row['raw_file_nme'],
            'success': result.get('success', False),
            'chunks': rows_inserted if result.get('success') else 0,
            'error': result.get('message', '') if not result.get('success') else None
        })
    except Exception as e:
        print(f"  ✗ {row['raw_file_nme']:40s} → ERROR: {str(e)}")
        ingestion_results.append({
            'file_id': row['ud_source_file_id'],
            'file_name': row['raw_file_nme'],
            'success': False,
            'chunks': 0,
            'error': str(e)
        })

print(f"\n✓ Processed {files_processed} files")
print(f"✓ Generated {total_chunks} chunks")
print(f"✓ Average chunks per file: {total_chunks / files_processed if files_processed > 0 else 0:.1f}")

2025-11-02 14:42:31.895 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:31.896 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:31.897 | INFO     | database.repository.content_repository:clear_chunks_for_file:1282 - Clearing all chunks for file ID: 15
2025-11-02 14:42:31.898 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:31.899 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:31.901 | DEBUG    | database.backends.sqlite.backend:execute_command:321 - Command affected 0 rows
2025-11-02 14:42:31.903 | DEBUG    | database.repository.base:execute_command:682 - Command affected 0 rows
2025-11-02 14:42:31.904 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:31.905 | DEBUG    | database.repository.base:transaction:371 - Transaction committe


  CHUNKING AND CHECKSUM COMPUTATION - HybridMarkdownChunker

  ✓ Employee_Handbook.pdf                    →   1 chunks
  ✓ IT_Security_Policy.pdf                   →   1 chunks


2025-11-02 14:42:31.966 | DEBUG    | database.repository.base:execute_command:682 - Command affected 0 rows
2025-11-02 14:42:31.969 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:31.971 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:31.972 | INFO     | database.repository.content_repository:clear_chunks_for_file:1290 - Deleted 0 chunks for file 17
2025-11-02 14:42:31.976 | INFO     | database.repository.content_repository:ingest_chunks:1159 - Ingesting 1 chunks in bulk
2025-11-02 14:42:31.979 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:31.981 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:31.985 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 1 rows into content_chunks
2025-11-02 14:42:31.986 | DEBUG    | database.backends.sqlite.backend:c

  ✓ Remote_Work_Policy.pdf                   →   1 chunks


2025-11-02 14:42:31.999 | DEBUG    | database.backends.sqlite.backend:execute_command:321 - Command affected 0 rows
2025-11-02 14:42:32.002 | DEBUG    | database.repository.base:execute_command:682 - Command affected 0 rows
2025-11-02 14:42:32.003 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:32.005 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:32.007 | INFO     | database.repository.content_repository:clear_chunks_for_file:1290 - Deleted 0 chunks for file 18
2025-11-02 14:42:32.007 | INFO     | database.repository.content_repository:ingest_chunks:1159 - Ingesting 1 chunks in bulk
2025-11-02 14:42:32.007 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:32.007 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:32.007 | INFO     | database.backends.sqlite.backend:ingest_datafra

...

  ✓ Payroll_Processing.pdf                   →   1 chunks


2025-11-02 14:42:32.254 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:32.254 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:32.259 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 1 rows into content_chunks
2025-11-02 14:42:32.261 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:32.261 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:32.261 | INFO     | database.repository.content_repository:ingest_chunks:1240 - Successfully ingested 1 chunks
2025-11-02 14:42:32.261 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:32.261 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:32.261 | INFO     | database.repository.content_repository:clear_chunks_for_

  ✓ Code_of_Conduct.pdf                      →   1 chunks
  ✓ Travel_Policy.pdf                        →   1 chunks
  ✓ Expense_Reimbursement.pdf                →   1 chunks
  ✓ Performance_Reviews.pdf                  →   1 chunks
⚠️  WARNING: File not found: data_ingestion\data\markdown\Parental_Leave_v1.md
  ✓ Health_Insurance.pdf                     →   1 chunks
⚠️  WARNING: File not found: data_ingestion\data\markdown\Vacation_Policy_v1.md


2025-11-02 14:42:32.369 | INFO     | database.repository.content_repository:ingest_chunks:1240 - Successfully ingested 1 chunks
2025-11-02 14:42:32.369 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:32.369 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:32.369 | INFO     | database.repository.content_repository:clear_chunks_for_file:1282 - Clearing all chunks for file ID: 14
2025-11-02 14:42:32.369 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:32.369 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:32.369 | DEBUG    | database.backends.sqlite.backend:execute_command:321 - Command affected 1 rows
2025-11-02 14:42:32.369 | DEBUG    | database.repository.base:execute_command:682 - Command affected 1 rows
2025-11-02 14:42:32.369 | DEBUG    | database.backends.sqlite.backend:commit:

  ✓ Email_Policy.pdf                         →   1 chunks
  ✓ Parking_Policy.pdf                       →   1 chunks

✓ Processed 20 files
✓ Generated 20 chunks
✓ Average chunks per file: 1.0


## Step 6: Initialize Detector with Repository Integration

**Key innovation:** Create detector with `AuditRepository` for integrated workflow.

In [7]:
from detection import ChecksumChangeDetector

print(f"\n{'='*80}")
print("  INITIALIZING CHANGE DETECTOR WITH REPOSITORY")
print(f"{'='*80}\n")

# Create detector with repository integration
detector = ChecksumChangeDetector.for_faq_updates(audit_repository=audit_repo)

print("✓ Detector created with repository integration")
print(f"  - Algorithm: {detector.config.checksum_algorithm}")
print(f"  - Similarity Threshold: {detector.config.similarity_threshold}")
print(f"  - LLM Diffs: {detector.config.compute_llm_diffs}")
print(f"  - Repository Mode: Enabled")
print(f"\n✅ Ready to detect changes!")
print(f"{'='*80}")

2025-11-02 14:42:32.470 | INFO     | detection.checksum_detector:__init__:255 - ChecksumChangeDetector initialized: algorithm=sha256, similarity_threshold=0.8, compute_llm_diffs=True, repository_mode=enabled



  INITIALIZING CHANGE DETECTOR WITH REPOSITORY

✓ Detector created with repository integration
  - Algorithm: sha256
  - Similarity Threshold: 0.8
  - LLM Diffs: True
  - Repository Mode: Enabled

✅ Ready to detect changes!


## Step 7: Detect Changes Using Repository Pattern

Process each file:
1. Load baseline data from database using `detector.load_previous_checksums()`
2. Load current data from database using `ContentRepository`
3. Detect changes using `detector.detect_changes()`
4. Store results using `detector.store_detection_results()`

In [8]:
from core.models.detection import ChangeType

print(f"\n{'='*80}")
print("  DETECTING CHANGES")
print(f"{'='*80}\n")

all_results = []

# 1. Load baseline data from database (using detector's integrated method)
all_previous_data = audit_repo.get_previous_checksums(since_timestamp=SINCE_DATE)
print(f"  Previous chunks: {len(all_previous_data)}")

# 2 Get all new checksums created since a specific date
all_new_data = audit_repo.get_new_checksums(
    since_timestamp=SINCE_DATE
)
print(f"  New chunks: {len(all_new_data)}")

# ===================================================================
for file_info in modified_files:
    file_name = file_info['raw_file_nme']
    print(f"Processing {file_name}:")
    
    # FIX 3: Filter GLOBAL data by file_name (matches code1's approach)
    new_data = {k: v for k, v in all_new_data.items() if v["file_name"] == file_name}
    previous_data = {k: v for k, v in all_previous_data.items() if v.get("file_name") == file_name}
    
    print(f"  Current chunks: {len(new_data)}")
    print(f"  Previous chunks: {len(previous_data)}")
    
    # Detect changes
    results = detector.detect_changes(
        file_name=file_name,
        current_checksums_data=new_data,
        previous_checksums_data=previous_data,
        detection_run_id=DETECTION_RUN_ID
    )
    
    all_results.extend(results)
    print(f"  Changes detected: {len(results)}")

print(f"\n{'='*80}")
print(f"Total changes detected: {len(all_results)}")
print(f"{'='*80}")


# # Process each modified file
# for file_info in modified_files:
#     file_name = file_info['raw_file_nme']
#     print(f"Processing {file_name}:")
    
#     # 1. Load baseline data from database (using detector's integrated method)
#     previous_data = detector.load_previous_checksums(file_name=file_name)
#     print(f"  Previous chunks: {len(previous_data)}")
    
#     # Get all new checksums created since a specific date
#     new_content = audit_repo.get_new_checksums(
#         file_name="policy.pdf",
#         since_timestamp="2025-11-02 10:24:29"
#     )
    
#     # Convert to detector format
#     new_data = {}
#     for chunk in current_chunks:
#         current_data[chunk['content_checksum']] = {
#             'text': chunk['chunk_text'],
#             'file_name': file_name
#         }
    
#     print(f"  Current chunks: {len(current_data)}")
    
#     # 3. Detect changes
#     results = detector.detect_changes(
#         file_name=file_name,
#         current_data=current_data,
#         previous_data=previous_data,
#         run_id=DETECTION_RUN_ID
#     )
    
#     all_results.extend(results)
#     print(f"  Changes detected: {len(results)}")

# print(f"\n{'='*80}")
# print(f"Total changes detected: {len(all_results)}")
# print(f"{'='*80}")

2025-11-02 14:42:32.496 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:32.498 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:32.498 | INFO     | database.repository.audit_repository:get_previous_checksums:827 - Retrieved 0 previous checksums (file_name=None, since_timestamp=2025-11-02 10:24:29)
2025-11-02 14:42:32.501 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 20 rows
2025-11-02 14:42:32.502 | DEBUG    | database.repository.base:execute_query:590 - Query returned 20 rows
2025-11-02 14:42:32.503 | INFO     | database.repository.audit_repository:get_new_checksums:937 - Retrieved 20 new checksums (file_name=None, since_timestamp=2025-11-02 10:24:29)
2025-11-02 14:42:32.505 | INFO     | detection.checksum_detector:detect_changes:294 - 🔍 Detecting changes for: Employee_Handbook.pdf
2025-11-02 14:42:32.506 | INFO     | detection.checksum_detector:d


  DETECTING CHANGES

  Previous chunks: 0
  New chunks: 20
Processing Employee_Handbook.pdf:
  Current chunks: 2
  Previous chunks: 0
  Changes detected: 2
Processing IT_Security_Policy.pdf:
  Current chunks: 2
  Previous chunks: 0
  Changes detected: 2
Processing Remote_Work_Policy.pdf:
  Current chunks: 2
  Previous chunks: 0


2025-11-02 14:42:32.544 | INFO     | detection.checksum_detector:detect_changes:309 -    New:       2 checksums
2025-11-02 14:42:32.545 | INFO     | detection.checksum_detector:detect_changes:310 -    Deleted:   0 checksums
2025-11-02 14:42:32.545 | INFO     | detection.checksum_detector:detect_changes:311 -    Unchanged: 0 checksums
2025-11-02 14:42:32.546 | INFO     | detection.checksum_detector:detect_changes:338 - ⚡ No similarity matching needed (no overlap between new/deleted)
2025-11-02 14:42:32.547 | INFO     | detection.checksum_detector:detect_changes:404 - ✅ Changes detected: 2
2025-11-02 14:42:32.548 | INFO     | detection.checksum_detector:detect_changes:405 -    New:      2
2025-11-02 14:42:32.548 | INFO     | detection.checksum_detector:detect_changes:406 -    Modified: 0
2025-11-02 14:42:32.549 | INFO     | detection.checksum_detector:detect_changes:407 -    Deleted:  0
2025-11-02 14:42:32.550 | INFO     | detection.checksum_detector:detect_changes:408 -    Unchanged: 0


  Changes detected: 2
Processing Travel_Policy.pdf:
  Current chunks: 2
  Previous chunks: 0
  Changes detected: 2
Processing Parental_Leave.pdf:
  Current chunks: 1
  Previous chunks: 0
  Changes detected: 1
Processing Workplace_Safety.pdf:
  Current chunks: 0
  Previous chunks: 0
  Changes detected: 0
Processing Health_Insurance.pdf:
  Current chunks: 2
  Previous chunks: 0
  Changes detected: 2
Processing Vacation_Policy.pdf:
  Current chunks: 0
  Previous chunks: 0
  Changes detected: 0
Processing Email_Policy.pdf:
  Current chunks: 2
  Previous chunks: 0
  Changes detected: 2
Processing Equipment_Return.pdf:
  Current chunks: 1
  Previous chunks: 0
  Changes detected: 1
Processing Employee_Handbook.pdf:
  Current chunks: 2
  Previous chunks: 0


2025-11-02 14:42:32.654 | INFO     | detection.checksum_detector:detect_changes:308 -    Previous:  0 checksums
2025-11-02 14:42:32.655 | INFO     | detection.checksum_detector:detect_changes:309 -    New:       2 checksums
2025-11-02 14:42:32.656 | INFO     | detection.checksum_detector:detect_changes:310 -    Deleted:   0 checksums
2025-11-02 14:42:32.658 | INFO     | detection.checksum_detector:detect_changes:311 -    Unchanged: 0 checksums
2025-11-02 14:42:32.658 | INFO     | detection.checksum_detector:detect_changes:338 - ⚡ No similarity matching needed (no overlap between new/deleted)
2025-11-02 14:42:32.659 | INFO     | detection.checksum_detector:detect_changes:404 - ✅ Changes detected: 2
2025-11-02 14:42:32.660 | INFO     | detection.checksum_detector:detect_changes:405 -    New:      2
2025-11-02 14:42:32.660 | INFO     | detection.checksum_detector:detect_changes:406 -    Modified: 0
2025-11-02 14:42:32.660 | INFO     | detection.checksum_detector:detect_changes:407 -    De

...

  Changes detected: 1
Processing Travel_Policy.pdf:
  Current chunks: 2
  Previous chunks: 0
  Changes detected: 2
Processing Expense_Reimbursement.pdf:
  Current chunks: 1
  Previous chunks: 0
  Changes detected: 1
Processing Performance_Reviews.pdf:
  Current chunks: 1
  Previous chunks: 0
  Changes detected: 1
Processing Parental_Leave.pdf:
  Current chunks: 1
  Previous chunks: 0
  Changes detected: 1
Processing Health_Insurance.pdf:
  Current chunks: 2
  Previous chunks: 0


2025-11-02 14:42:32.842 | INFO     | detection.checksum_detector:detect_changes:406 -    Modified: 0
2025-11-02 14:42:32.842 | INFO     | detection.checksum_detector:detect_changes:407 -    Deleted:  0
2025-11-02 14:42:32.845 | INFO     | detection.checksum_detector:detect_changes:408 -    Unchanged: 0
2025-11-02 14:42:32.845 | INFO     | detection.checksum_detector:detect_changes:409 - ⏱️  Processing time: 15.70ms
2025-11-02 14:42:32.845 | INFO     | detection.checksum_detector:detect_changes:294 - 🔍 Detecting changes for: Vacation_Policy.pdf
2025-11-02 14:42:32.847 | INFO     | detection.checksum_detector:detect_changes:295 -    Detection run: run_20251102_144231_400e5290
2025-11-02 14:42:32.847 | INFO     | detection.checksum_detector:detect_changes:306 - 📊 Checksum analysis for Vacation_Policy.pdf:
2025-11-02 14:42:32.849 | INFO     | detection.checksum_detector:detect_changes:307 -    Current:   0 checksums
2025-11-02 14:42:32.850 | INFO     | detection.checksum_detector:detect_ch

  Changes detected: 2
Processing Vacation_Policy.pdf:
  Current chunks: 0
  Previous chunks: 0


2025-11-02 14:42:32.857 | INFO     | detection.checksum_detector:detect_changes:407 -    Deleted:  0
2025-11-02 14:42:32.858 | INFO     | detection.checksum_detector:detect_changes:408 -    Unchanged: 0
2025-11-02 14:42:32.859 | INFO     | detection.checksum_detector:detect_changes:409 - ⏱️  Processing time: 7.59ms
2025-11-02 14:42:32.863 | INFO     | detection.checksum_detector:detect_changes:294 - 🔍 Detecting changes for: Email_Policy.pdf
2025-11-02 14:42:32.865 | INFO     | detection.checksum_detector:detect_changes:295 -    Detection run: run_20251102_144231_400e5290
2025-11-02 14:42:32.866 | INFO     | detection.checksum_detector:detect_changes:306 - 📊 Checksum analysis for Email_Policy.pdf:
2025-11-02 14:42:32.868 | INFO     | detection.checksum_detector:detect_changes:307 -    Current:   2 checksums
2025-11-02 14:42:32.869 | INFO     | detection.checksum_detector:detect_changes:308 -    Previous:  0 checksums
2025-11-02 14:42:32.870 | INFO     | detection.checksum_detector:detec

  Changes detected: 0
Processing Email_Policy.pdf:
  Current chunks: 2
  Previous chunks: 0
  Changes detected: 2
Processing Parking_Policy.pdf:
  Current chunks: 1
  Previous chunks: 0
  Changes detected: 1

Total changes detected: 33


## Step 8: Store Results Using Repository

Use detector's integrated `store_detection_results()` method.

In [9]:
all_results

[ContentChange(old_checksum=None, new_checksum='b54c22ec37e28c72f242c5c96ba13a0d582cf14ad4bba7b39f24466911546ca1', change_type=<ChangeType.NEW_CONTENT: 'new_content'>, detected_at=datetime.datetime(2025, 11, 2, 14, 42, 32, 511562), file_name='Employee_Handbook.pdf', page_number=0, old_content=None, new_content='# Employee Handbook 2024\n\n## Time Off Policy\n\nEmployees are entitled to 10 sick days per calendar year. Sick days do not roll over to the next year.\n\n## Work Schedule\n\nStandard work hours are Monday through Friday, 9:00 AM to 5:00 PM.\n\n## Benefits Overview\n\nAll full-time employees are eligible for health insurance, dental coverage, and vision benefits after 90 days of employment.\n', similarity_score=0.0, llm_friendly_diff=None),
 ContentChange(old_checksum=None, new_checksum='f3bbae03d26ce982636aaa5552ad07e8735c7dd28f36d3cdcf2310f9f6168479', change_type=<ChangeType.NEW_CONTENT: 'new_content'>, detected_at=datetime.datetime(2025, 11, 2, 14, 42, 32, 511562), file_name

In [10]:
if len(all_results) > 0:
    print("\n💾 Storing detection results...")
    
    # Store using detector's integrated method
    store_result = detector.store_detection_results(
        detection_results=all_results,
        detection_run_id=DETECTION_RUN_ID
    )
    
    print(f"✅ Stored {store_result['rows_inserted']} records")
    print(f"   Success: {store_result['success']}")
    
    # Verify results using AuditRepository
    changes_by_type = {}
    for result in all_results:
        change_type = result.change_type.value
        changes_by_type[change_type] = changes_by_type.get(change_type, 0) + 1
    
    print("\n📊 Change Summary:")
    for change_type, count in changes_by_type.items():
        print(f"  {change_type}: {count}")
else:
    print("\n⚠️ No changes detected to store")


💾 Storing detection results...

2025-11-02 14:42:32.929 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:32.929 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:32.939 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 33 rows into content_change_log
2025-11-02 14:42:32.941 | INFO     | database.repository.audit_repository:log_changes_bulk:406 - Bulk inserted 33 changes for detection run: run_20251102_144231_400e5290
2025-11-02 14:42:32.942 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:32.942 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully



✅ Stored 33 records
   Success: True

📊 Change Summary:
  new_content: 33


## Step 9: View Results Using AuditRepository

Use repository methods to query and analyze results.

In [11]:
# Get changes for this run using AuditRepository
run_changes = audit_repo.get_changes_by_run(DETECTION_RUN_ID, include_unchanged=True)

print(f"\n📋 Retrieved {len(run_changes)} changes from database")

if len(run_changes) > 0:
    # Convert to DataFrame for display
    changes_df = pd.DataFrame([
        {
            'change_id': c.change_id,
            'file_name': c.file_name,
            'change_type': c.change_type.value if c.change_type else 'unknown',
            'similarity_score': c.similarity_score,
            'requires_regen': c.requires_faq_regeneration,
            'detection_timestamp': c.detection_timestamp
        }
        for c in run_changes
    ])
    
    display(changes_df.head(20))

2025-11-02 14:42:32.955 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 33 rows
2025-11-02 14:42:32.957 | DEBUG    | database.repository.base:execute_query:590 - Query returned 33 rows
2025-11-02 14:42:32.958 | INFO     | database.repository.audit_repository:get_changes_by_run:581 - Retrieved 33 changes for run: run_20251102_144231_400e5290



📋 Retrieved 33 changes from database


change_id                  file_name  change_type  similarity_score  \
0          21         Benefits_Guide.pdf  new_content               0.0   
1          23        Code_of_Conduct.pdf  new_content               0.0   
2          12           Email_Policy.pdf  new_content               0.0   
3          13           Email_Policy.pdf  new_content               0.0   
4          31           Email_Policy.pdf  new_content               0.0   
5          32           Email_Policy.pdf  new_content               0.0   
6           1      Employee_Handbook.pdf  new_content               0.0   
7           2      Employee_Handbook.pdf  new_content               0.0   
8          15      Employee_Handbook.pdf  new_content               0.0   
9          16      Employee_Handbook.pdf  new_content               0.0   
10         14       Equipment_Return.pdf  new_content               0.0   
11         26  Expense_Reimbursement.pdf  new_content               0.0   
12         10       Health_Insurance.pdf  new_content               0.0   
13         11       Health_Insurance.pdf  new_content               0.0   
14         29       Health_Insurance.pdf  new_content               0.0   
15         30       Health_Insurance.pdf  new_content               0.0   
16          3     IT_Security_Policy.pdf  new_content               0.0   
17          4     IT_Security_Policy.pdf  new_content               0.0   
18         17     IT_Security_Policy.pdf  new_content               0.0   
19         18     IT_Security_Policy.pdf  new_content               0.0   

    requires_regen         detection_timestamp  
0             True  2025-11-02 14:42:32.921233  
1             True  2025-11-02 14:42:32.921233  
2             True  2025-11-02 14:42:32.921233  
3             True  2025-11-02 14:42:32.921233  
4             True  2025-11-02 14:42:32.921233  
5             True  2025-11-02 14:42:32.921233  
6             True  2025-11-02 14:42:32.921233  
7             True  2025-11-02 14:42:32.921233  
8             True  2025-11-02 14:42:32.921233  
9             True  2025-11-02 14:42:32.921233  
10            True  2025-11-02 14:42:32.921233  
11            True  2025-11-02 14:42:32.921233  
12            True  2025-11-02 14:42:32.921233  
13            True  2025-11-02 14:42:32.921233  
14            True  2025-11-02 14:42:32.921233  
15            True  2025-11-02 14:42:32.921233  
16            True  2025-11-02 14:42:32.921233  
17            True  2025-11-02 14:42:32.921233  
18            True  2025-11-02 14:42:32.921233  
19            True  2025-11-02 14:42:32.921233

## Step 10: Get Detection Run Summary

Use `AuditRepository` to get comprehensive statistics.

In [12]:
# Get run summary using AuditRepository
summary = audit_repo.get_run_summary(detection_run_id=DETECTION_RUN_ID)

if summary:
    summary_dict = summary[0]
    
    print("="*80)
    print("✅ DETECTION COMPLETE")
    print("="*80)
    print(f"Environment: {'Databricks' if IS_DATABRICKS else 'SQLite'}")
    print(f"Run ID: {DETECTION_RUN_ID}")
    print(f"\nRun Summary:")
    print(f"  Total Changes: {summary_dict['total_changes_detected']}")
    print(f"  Unique Files: {summary_dict['unique_files_changed']}")
    print(f"  FAQs at Risk: {summary_dict['total_faqs_at_risk']}")
    print(f"  Questions Affected: {summary_dict['questions_actually_affected']}")
    print(f"  Answers Affected: {summary_dict['answers_actually_affected']}")
    print(f"  FAQs Saved: {summary_dict['faqs_saved_from_regeneration']}")
    print(f"  Savings: {summary_dict['savings_percentage']}%")
    print(f"  Avg Similarity: {summary_dict['avg_similarity_score']}")
    print("="*80)
else:
    print("\n⚠️ No summary available for this run")

2025-11-02 14:42:32.982 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:32.983 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:32.984 | INFO     | database.repository.audit_repository:get_run_summary:717 - Retrieved summaries for 1 detection run(s)


✅ DETECTION COMPLETE
Environment: SQLite
Run ID: run_20251102_144231_400e5290

Run Summary:
  Total Changes: 33
  Unique Files: 14
  FAQs at Risk: 0
  Questions Affected: 0
  Answers Affected: 0
  FAQs Saved: 0
  Savings: None%
  Avg Similarity: 0.0


## Summary

### What We Did

1. ✅ Initialized backend and repositories using `BackendFactory` and `AuditRepository`
2. ✅ Loaded modified content using `ContentRepository`
3. ✅ Chunked and stored content using repository methods
4. ✅ Created detector with repository integration
5. ✅ Detected changes using `detector.detect_changes()`
6. ✅ Stored results using `detector.store_detection_results()`
7. ✅ Queried results using `AuditRepository` methods

### Benefits of Repository Pattern

- **Database-Agnostic**: Same code works for SQLite and Databricks
- **Cleaner API**: No custom adapter classes needed
- **Integrated Workflow**: All operations through one detector object
- **Better Separation**: Detection logic is pure, database logic is in repository
- **Easier Testing**: Mock repositories instead of databases

### Next Steps

1. Review changes requiring FAQ regeneration
2. Run FAQ generation workflows for affected FAQs
3. Update FAQ audit logs
4. Monitor savings metrics over time

---

**For more information**, see:
- [REPOSITORY_INTEGRATION_GUIDE.md](detection/REPOSITORY_INTEGRATION_GUIDE.md)
- [MIGRATION_PLAN_SQLITE_TO_DATABRICKS.md](MIGRATION_PLAN_SQLITE_TO_DATABRICKS.md)

In [2]:
import sqlite3
import pandas as pd

# Database path (adjust to your actual database location)
db_path = r'C:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db'

# CSV file path
csv_path = r'C:\Users\praka\projects\FAQ_combined\faq_update\data_ingestion\data\7_content_change_log.csv'

# Connect to database
conn = sqlite3.connect(db_path)

# Read CSV and load into database
df = pd.read_csv(csv_path)
df.to_sql('content_change_log', conn, if_exists='append', index=False)

# Verify the import
cursor = conn.cursor()
count = cursor.execute("SELECT COUNT(*) FROM content_change_log").fetchone()[0]
print(f"✓ Loaded {count} records into content_change_log table")

# Close connection
conn.close()

✓ Loaded 20 records into content_change_log table
